# Credit:

[Leonardo Ferreira](https://www.kaggle.com/kabure) and [his work](https://www.kaggle.com/kabure/financial-hedging-eda-interactive-plots)

### Imports

In [1]:
# Load the librarys
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from plotly import tools
import plotly.offline as py
from plotly.offline import init_notebook_mode, iplot, plot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

import warnings
warnings.filterwarnings('ignore')

### Load data

In [2]:
county = pd.read_csv("../../../data/census/census_county.csv")

FileNotFoundError: File b'../../../data/census/census_county.csv' does not exist

# Races

#### setup

In [37]:
races = ['Hispanic', 'White', 'Black', 'Native', 'Asian', 'Pacific']

for race in races:
    county[race + "_pop"] = (county[race] * county.TotalPop) / 100

### Races - by county

In [44]:
datas

[              State          County  Hispanic_pop                       Place
 0        California     Los Angeles   4838503.016     Los Angeles, California
 1             Texas          Harris   1812246.592               Harris, Texas
 2           Florida      Miami-Dade   1731211.552         Miami-Dade, Florida
 3          Illinois            Cook   1293389.071              Cook, Illinois
 4           Arizona        Maricopa   1209461.043           Maricopa, Arizona
 5        California       Riverside   1080075.040       Riverside, California
 6             Texas           Bexar   1078871.682                Bexar, Texas
 7        California  San Bernardino   1070426.959  San Bernardino, California
 8        California          Orange   1065695.598          Orange, California
 9        California       San Diego   1060398.584       San Diego, California
 10            Texas          Dallas    969151.170               Dallas, Texas
 11         New York           Bronx    779882.922  

In [48]:
races = ['Hispanic', 'White', 'Black', 'Native', 'Asian', 'Pacific']

### Create individual figures
fig = tools.make_subplots(rows=2, cols=2, subplot_titles=('Highest Population',     'Lowest Population',
                                                          'Highest Representation', 'Lowest Representation'))

###################################### Population ######################################

datas = []
for race in races:
    data = county.groupby(["State", "County"])[race + "_pop"].sum().map(int).sort_values(ascending=False)
    data = data.reset_index()
    data["Place"] = data["County"] + ", " + data["State"]
    data = data.rename(columns={race + "_pop": "Values"})
    datas.append(data)
    

for data in datas:
    trace = go.Bar(
                    x=data.head(10).Place,
                    y=data.head(10).Values, 
                    showlegend=False
                )
    fig.append_trace(trace, 1, 1)
    
for data in datas:
    trace = go.Bar(
                    x=data.tail(10).Place,
                    y=data.tail(10).Values, 
                    showlegend=False
                )
    fig.append_trace(trace, 1, 2)
    
###################################### Representation ######################################

datas = []
for race in races:
    data = county.groupby(["State", "County"])[race].median().sort_values(ascending=False)
    data = data.reset_index()
    data["Place"] = data["County"] + ", " + data["State"]
    data = data.rename(columns={race: "Values"})
    datas.append(data)

for data in datas:
    trace = go.Bar(
                    x=data.head(10).Place,
                    y=data.head(10).Values, 
                    showlegend=False
                )
    fig.append_trace(trace, 2, 1)
    
for data in datas:
    trace = go.Bar(
                    x=data.tail(10).Place,
                    y=data.tail(10).Values, 
                    showlegend=False
                )
    fig.append_trace(trace, 2, 2)

### Create buttons for drop down menu
buttons = []
for i, label in enumerate(races):
    visibility = [i==j for j in range(len(races))]
    button = dict(
                 label =  label,
                 method = 'update',
                 args = [{'visible': visibility},
                     {'title': label}])
    buttons.append(button)

### Create menu
updatemenus = list([
    dict(active=-1,
         x=-0.15,
         buttons=buttons
    )
])

### Final figure edits
fig['layout']['title'] = 'Racial Population in Counties'
fig['layout']['showlegend'] = False
fig['layout']['updatemenus'] = updatemenus
fig['layout'].update(height=800, width=1000)

iplot(fig, filename='dropdown')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]



### Races - by state

In [41]:
races = ['Hispanic', 'White', 'Black', 'Native', 'Asian', 'Pacific']

datas = []
for race in races:
    data = county.groupby("State")[race + "_pop"].sum().sort_values(ascending=False)
    datas.append(data)

### Create individual figures
fig = tools.make_subplots(rows=2, cols=2, subplot_titles=('1st Quartile', '2nd Quartile',
                                                          '3rd Quartile', '4th Quartile'))

for i in range(4):
    for data in datas:
        start_i = 13 * i
        end_i   = start_i + 13
        
        trace = go.Bar(
                        x=data.iloc[start_i: end_i].index,
                        y=data.iloc[start_i: end_i].values, 
                        showlegend=False
                    )
        
        row_num = 1 + (i // 2)
        col_num = 1 + (i % 2)
        fig.append_trace(trace, row_num, col_num)

### Create buttons for drop down menu
buttons = []
for i, label in enumerate(races):
    visibility = [i==j for j in range(len(races))]
    button = dict(
                 label =  label,
                 method = 'update',
                 args = [{'visible': visibility},
                     {'title': label}])
    buttons.append(button)

updatemenus = list([
    dict(active=-1,
         x=-0.15,
         buttons=buttons
    )
])

fig['layout']['title'] = 'Racial Population in all 52 States'
fig['layout']['showlegend'] = False
fig['layout']['updatemenus'] = updatemenus
fig['layout'].update(height=800, width=1000)

iplot(fig, filename='dropdown')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]



#### Delete unneeded variables

In [ ]:
del races, race, race_pops, race_reps
races = ['Hispanic', 'White', 'Black', 'Native', 'Asian', 'Pacific']
county = county.drop([race + "_pop" for race in races], axis=1)

### Career and Sector Plotly

In [34]:
careers = ['Professional', 'Service', 'Office', 'Construction', 'Production']

### Create individual figures
car_fig = tools.make_subplots(rows=1, cols=2, subplot_titles=('Highest', 'Lowest',))

###################################### Career plots ######################################

datas = []
for career in careers:
    data = county.groupby(["State", "County"])[career].median().sort_values(ascending=False)
    data = data.reset_index()
    data["Place"] = data["County"] + ", " + data["State"]
    data = data.rename(columns={career: "Values"})
    datas.append(data)
    
for data in datas:
    trace = go.Bar(
                    x=data.head(10).Place,
                    y=data.head(10).Values, 
                    showlegend=False
                )
    car_fig.append_trace(trace, 1, 1)

for data in datas:
    trace = go.Bar(
                    x=data.tail(10).Place,
                    y=data.tail(10).Values, 
                    showlegend=False
                )
    car_fig.append_trace(trace, 1, 2)

### Create buttons for drop down menu
buttons = []
for i, label in enumerate(careers):
    visibility = [i==j for j in range(len(careers))]
    button = dict(
                 label =  label,
                 method = 'update',
                 args = [{'visible': visibility},
                     {'title': label}])
    buttons.append(button)

### Create menu
updatemenus = list([
    dict(active=-1,
         x=-0.15,
         buttons=buttons
    )
])

### Final figure edits
car_fig['layout']['title'] = 'Career'
car_fig['layout']['showlegend'] = False
car_fig['layout']['updatemenus'] = updatemenus

iplot(car_fig, filename='dropdown')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



### In progress - I want to have subplots within subplots.

Also the subplots each have a menu to change either between careers or sectors

In [36]:
careers = ['Professional', 'Service', 'Office', 'Construction', 'Production']
sectors = ['PrivateWork', 'PublicWork', 'SelfEmployed', 'FamilyWork']

### Create individual figures
main_fig = tools.make_subplots(rows=2, cols=1, subplot_titles=('Career', 'Sector',))
car_fig = tools.make_subplots(rows=1, cols=2, subplot_titles=('Highest', 'Lowest',))
sec_fig = tools.make_subplots(rows=1, cols=2, subplot_titles=('Highest', 'Lowest',))

###################################### Career plots ######################################

datas = []
for career in careers:
    data = county.groupby(["State", "County"])[career].median().sort_values(ascending=False)
    data = data.reset_index()
    data["Place"] = data["County"] + ", " + data["State"]
    data = data.rename(columns={career: "Values"})
    datas.append(data)
    
for data in datas:
    trace = go.Bar(
                    x=data.head(10).Place,
                    y=data.head(10).Values, 
                    showlegend=False
                )
    car_fig.append_trace(trace, 1, 1)

for data in datas:
    trace = go.Bar(
                    x=data.tail(10).Place,
                    y=data.tail(10).Values, 
                    showlegend=False
                )
    car_fig.append_trace(trace, 1, 2)

### Create buttons for drop down menu
buttons = []
for i, label in enumerate(careers):
    visibility = [i==j for j in range(len(careers))]
    button = dict(
                 label =  label,
                 method = 'update',
                 args = [{'visible': visibility},
                     {'title': label}])
    buttons.append(button)

### Create menu
updatemenus = list([
    dict(active=-1,
         x=-0.15,
         buttons=buttons
    )
])

### Final figure edits
car_fig['layout']['title'] = 'Career'
car_fig['layout']['showlegend'] = False
car_fig['layout']['updatemenus'] = updatemenus


###################################### Sector plots ######################################

datas = []
for sector in sectors:
    data = county.groupby(["State", "County"])[sector].median().sort_values(ascending=False)
    data = data.reset_index()
    data["Place"] = data["County"] + ", " + data["State"]
    data = data.rename(columns={sector: "Values"})
    datas.append(data)
    
for data in datas:
    trace = go.Bar(
                    x=data.head(10).Place,
                    y=data.head(10).Values, 
                    showlegend=False
                )
    sec_fig.append_trace(trace, 1, 1)

for data in datas:
    trace = go.Bar(
                    x=data.tail(10).Place,
                    y=data.tail(10).Values, 
                    showlegend=False
                )
    sec_fig.append_trace(trace, 1, 2)

### Create buttons for drop down menu
buttons = []
for i, label in enumerate(sectors):
    visibility = [i==j for j in range(len(sectors))]
    button = dict(
                 label =  label,
                 method = 'update',
                 args = [{'visible': visibility},
                     {'title': label}])
    buttons.append(button)


### Create menu
updatemenus = list([
    dict(active=-1,
         x=-0.15,
         buttons=buttons
    )
])

### Final figure edits
sec_fig['layout']['title'] = 'Career'
sec_fig['layout']['showlegend'] = False
sec_fig['layout']['updatemenus'] = updatemenus

###################################### Final plot ######################################
main_fig.append_trace(car_fig, 1, 1)
main_fig.append_trace(sec_fig, 2, 1)

del careers, sectors, car_fig, car, sec_fig, sec
iplot(main_fig, filename='dropdown')

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



PlotlyDictKeyError: 'xaxis' is not allowed in 'figure'

Path To Error: ['xaxis']

Valid attributes for 'figure' at path [] under parents []:

    ['data', 'frames', 'layout']

Run `<figure-object>.help('attribute')` on any of the above.
'<figure-object>' is the object at []

### Testing plotly - State

In [17]:
transportations = ['Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp']

datas = []
for tran in transportations:
    county["trans"] = county.TotalPop * county[tran]
    data = county.groupby("State")["trans"].sum() / county.groupby("State")["TotalPop"].sum()
    datas.append(data.sort_values(ascending=False))

### Create individual figures
fig = tools.make_subplots(rows=2, cols=2, subplot_titles=('1st Quartile', '2nd Quartile',
                                                          '3rd Quartile', '4th Quartile'))

for i in range(4):
    for data in datas:
        start_i = 13 * i
        end_i   = start_i + 13
        
        trace = go.Bar(
                        x=data.iloc[start_i: end_i].index,
                        y=data.iloc[start_i: end_i].values, 
                        showlegend=False
                    )
        
        row_num = 1 + (i // 2)
        col_num = 1 + (i % 2)
        fig.append_trace(trace, row_num, col_num)

### Create buttons for drop down menu
buttons = []
for i, label in enumerate(transportations):
    visibility = [i==j for j in range(len(transportations))]
    button = dict(
                 label =  label,
                 method = 'update',
                 args = [{'visible': visibility},
                     {'title': label}])
    buttons.append(button)

updatemenus = list([
    dict(active=-1,
         x=-0.15,
         buttons=buttons
    )
])

fig['layout']['title'] = 'Transportation across the states'
fig['layout']['showlegend'] = False
fig['layout']['updatemenus'] = updatemenus
fig['layout'].update(height=800, width=1000)

iplot(fig, filename='dropdown')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]



### Testing plotly - County

In [3]:
transportations = ['Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp']

datas = []
for tran in transportations:
    datas.append(county.groupby(["State", "County"])[tran].median().sort_values(ascending=False).head(10))

traces = []

for data in datas:
    traces.append(go.Box(
                            x=data.index,
                            y=data.values, 
                            showlegend=False
                        ))
buttons = []

for i, tran in enumerate(transportations):
    visibility = [i==j for j in range(len(transportations))]
    button = dict(
                 label =  tran,
                 method = 'update',
                 args = [{'visible': visibility},
                     {'title': 'Top counties for {}'.format(tran)}])
    buttons.append(button)

updatemenus = list([
    dict(active=-1,
         x=-0.15,
         buttons=buttons
    )
])

layout = dict(title='Counties with most popular transportation methods', 
              showlegend=False,
              updatemenus=updatemenus)

fig = dict(data=traces, layout=layout)

iplot(fig, filename='dropdown')